# Trino Connection Demo

This notebook demonstrates how to connect to Trino using the `camber.trino.connect` function.

This function is designed to securely read credentials from your Camber environment variables, making it easy to connect to your data warehouse without hard-coding sensitive information.

**Prerequisites:**

Before you begin, ensure you have set your Trino credentials in https://app.cambercloud.com/data-connectors

In [1]:
# Import necessary libraries
import os
import pandas as pd
import camber.trino as trino
from camber.exceptions import CamberEnvironmentException

## 1. Establish Connection

Now, we'll call the `trino.connect()` function. It will automatically read all the environment variables we just checked and attempt to establish a connection.

In [2]:
conn = None
try:
    conn = trino.connect()
    print("Connection to Trino successful!")
    
except CamberEnvironmentException as e:
    print(f"Connection Failed: {e}")
    print("Please ensure your Trino environment variables are set correctly.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Connection to Trino successful!


## 2. Run a Simple Query

If the connection was successful, we can run a simple query to verify that everything is working.

In [3]:
if conn:
    try:
        cursor = conn.cursor()
        cursor.execute("SELECT CURRENT_USER, CURRENT_CATALOG, CURRENT_SCHEMA")
        
        # Fetch the result
        result = cursor.fetchone()
        print(f"User: {result[0]}")
        print(f"Catalog: {result[1]}")
        print(f"Schema: {result[2]}")
        
    except Exception as e:
        print(f"Query failed: {e}")
    finally:
        # Always close the cursor
        if 'cursor' in locals():
            cursor.close()
else:
    print("Skipping query: Connection object 'conn' was not created.")

User: info@cambercloud.com
Catalog: sample
Schema: demo


## 3. Fetch Data with Pandas

The connection object is compatible with Pandas' `read_sql` function. Let's list the available catalogs.

In [5]:
if conn:
    try:
        df = pd.read_sql("SHOW CATALOGS", conn)
        print("Successfully fetched catalogs:")
        display(df)
    except Exception as e:
        print(f"Pandas read_sql failed: {e}")
else:
    print("Skipping query: Connection object 'conn' was not created.")

/tmp/ipykernel_415/3140598778.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SHOW CATALOGS", conn)


Successfully fetched catalogs:


,Catalog
0,galaxy
1,sample
2,system


## 4. Close the Connection

Finally, it's important to close the connection when you're done to free up resources.

In [6]:
if conn:
    try:
        conn.close()
        print("Trino connection closed.")
    except Exception as e:
        print(f"Error closing connection: {e}")
else:
    print("No connection to close.")

Trino connection closed.
